In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM ,Dropout
from keras.optimizers import SGD, RMSprop, Adam, Nadam
from keras.callbacks import EarlyStopping

from sklearn import metrics
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
print(tf.__version__)

2.3.0


In [3]:
n_timesteps, n_features = 4, 36

raw_data = pd.read_csv('data_under_scene.csv')
dataset = raw_data.values

X = dataset[:, 0:36].astype(float)
Y = dataset[:, 36].astype(int)

print(X.shape)
print(Y.shape)

blocks = int(len(X) / n_timesteps)
X = np.array(np.split(X, blocks))

blocks = int(len(Y) / n_timesteps)
Y = np.array(np.split(Y, blocks))

Y = np.array(    
    [[list(set(item))[0]] for item in Y]
,dtype=np.int32)

#one hot encoding for Y

### Universal list of classes
classes = raw_data["class"].unique()
n_classes = len(classes)
### map each color to an integer
mapping = {}
for x in range(len(classes)):
  mapping[classes[x]] = x

# integer representation
for x in range(len(Y)):
  Y[x][0] = mapping[Y[x][0]]

Y = to_categorical(Y)
print(f"classes : {classes}")
print(Y)
print(Y.shape)

(4100, 36)
(4100,)
classes : [0 1 2 3 4]
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]
(1025, 5)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=9)

In [5]:
model = Sequential()


model.add(LSTM(units = 32, return_sequences = True,input_shape = (n_timesteps,n_features)))
model.add(Dropout(.4))

for i in range(2):
  model.add(LSTM(units = 32, return_sequences = True))
  model.add(Dropout(.4))

model.add(LSTM(units = 32))
model.add(Dense(26, activation='relu'))
model.add(Dense(units = n_classes, activation='softmax'))



optimizer = Adam(lr=.0001, beta_1=0.9, beta_2=0.999, clipnorm=1.0)


#compile and fit the model
model.compile(
    optimizer = optimizer, 
    metrics=['accuracy'],
    loss='categorical_crossentropy'
)

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 4, 32)             8832      
_________________________________________________________________
dropout (Dropout)            (None, 4, 32)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 4, 32)             8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 32)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 32)             8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 32)             0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                8

In [7]:
model.fit(X_train, y_train, batch_size=64,
          epochs=200,
          validation_data=(X_test, y_test),
          callbacks=[EarlyStopping(patience=10, restore_best_weights=True)])

model.save("lstm_model.h5")

Epoch 1/200
13/13 [==============================] - 2s 130ms/step - loss: 1.6089 - accuracy: 0.2159 - val_loss: 1.6080 - val_accuracy: 0.2195
Epoch 2/200
13/13 [==============================] - 0s 12ms/step - loss: 1.6082 - accuracy: 0.2354 - val_loss: 1.6072 - val_accuracy: 0.2634
Epoch 3/200
13/13 [==============================] - 0s 10ms/step - loss: 1.6077 - accuracy: 0.2378 - val_loss: 1.6063 - val_accuracy: 0.2634
Epoch 4/200
13/13 [==============================] - 0s 12ms/step - loss: 1.6066 - accuracy: 0.2415 - val_loss: 1.6053 - val_accuracy: 0.2634
Epoch 5/200
13/13 [==============================] - 0s 10ms/step - loss: 1.6061 - accuracy: 0.2415 - val_loss: 1.6043 - val_accuracy: 0.2634
Epoch 6/200
13/13 [==============================] - 0s 18ms/step - loss: 1.6048 - accuracy: 0.2415 - val_loss: 1.6031 - val_accuracy: 0.2634
Epoch 7/200
13/13 [==============================] - 0s 17ms/step - loss: 1.6036 - accuracy: 0.2415 - val_loss: 1.6016 - val_accuracy: 0.2634
Epoch

13/13 [==============================] - 0s 10ms/step - loss: 0.6810 - accuracy: 0.7049 - val_loss: 0.6186 - val_accuracy: 0.6878
Epoch 59/200
13/13 [==============================] - 0s 10ms/step - loss: 0.6711 - accuracy: 0.7085 - val_loss: 0.6089 - val_accuracy: 0.6878
Epoch 60/200
13/13 [==============================] - 0s 11ms/step - loss: 0.6599 - accuracy: 0.7049 - val_loss: 0.6021 - val_accuracy: 0.7024
Epoch 61/200
13/13 [==============================] - 0s 12ms/step - loss: 0.6574 - accuracy: 0.7220 - val_loss: 0.5943 - val_accuracy: 0.7073
Epoch 62/200
13/13 [==============================] - 0s 10ms/step - loss: 0.6556 - accuracy: 0.7171 - val_loss: 0.5818 - val_accuracy: 0.7512
Epoch 63/200
13/13 [==============================] - 0s 10ms/step - loss: 0.6342 - accuracy: 0.7366 - val_loss: 0.5709 - val_accuracy: 0.7659
Epoch 64/200
13/13 [==============================] - 0s 10ms/step - loss: 0.6284 - accuracy: 0.7354 - val_loss: 0.5635 - val_accuracy: 0.7902
Epoch 65/200

13/13 [==============================] - 0s 10ms/step - loss: 0.3976 - accuracy: 0.8256 - val_loss: 0.3211 - val_accuracy: 0.8732
Epoch 116/200
13/13 [==============================] - 0s 10ms/step - loss: 0.3822 - accuracy: 0.8378 - val_loss: 0.3176 - val_accuracy: 0.8683
Epoch 117/200
13/13 [==============================] - 0s 10ms/step - loss: 0.3914 - accuracy: 0.8439 - val_loss: 0.3149 - val_accuracy: 0.8780
Epoch 118/200
13/13 [==============================] - 0s 10ms/step - loss: 0.3997 - accuracy: 0.8427 - val_loss: 0.3130 - val_accuracy: 0.8732
Epoch 119/200
13/13 [==============================] - 0s 10ms/step - loss: 0.3762 - accuracy: 0.8610 - val_loss: 0.3109 - val_accuracy: 0.8829
Epoch 120/200
13/13 [==============================] - 0s 10ms/step - loss: 0.3971 - accuracy: 0.8378 - val_loss: 0.3091 - val_accuracy: 0.8683
Epoch 121/200
13/13 [==============================] - 0s 10ms/step - loss: 0.3938 - accuracy: 0.8488 - val_loss: 0.3109 - val_accuracy: 0.8683
Epoch 

Epoch 172/200
13/13 [==============================] - 0s 11ms/step - loss: 0.3247 - accuracy: 0.8805 - val_loss: 0.2593 - val_accuracy: 0.8878
Epoch 173/200
13/13 [==============================] - 0s 12ms/step - loss: 0.3125 - accuracy: 0.8854 - val_loss: 0.2645 - val_accuracy: 0.8878
Epoch 174/200
13/13 [==============================] - 0s 14ms/step - loss: 0.3167 - accuracy: 0.8768 - val_loss: 0.2613 - val_accuracy: 0.8878
Epoch 175/200
13/13 [==============================] - 0s 14ms/step - loss: 0.3313 - accuracy: 0.8732 - val_loss: 0.2564 - val_accuracy: 0.8878
Epoch 176/200
13/13 [==============================] - 0s 15ms/step - loss: 0.3186 - accuracy: 0.8878 - val_loss: 0.2565 - val_accuracy: 0.8976
Epoch 177/200
13/13 [==============================] - 0s 14ms/step - loss: 0.3267 - accuracy: 0.8720 - val_loss: 0.2555 - val_accuracy: 0.8927
Epoch 178/200
13/13 [==============================] - 0s 14ms/step - loss: 0.3163 - accuracy: 0.8671 - val_loss: 0.2582 - val_accuracy: